In [2]:
import re
import os
import pandas as pd
import numpy as np
import subprocess
from subprocess import Popen
import codecs

def isHeader(row):
    #check if header by regex searching for timestamp e.g. [00:00:00.01]
    pat = re.compile("\d\d:\d\d:\d\d.\d\d(?=\d)")
    inq_time = pat.search(row)
    if inq_time:
        return False
    return True
    
def reformat(file_name):
    save_name= file_name[0:6]+".txt"
    xl = pd.ExcelFile(file_name)

    df = xl.parse("Sheet1", index_col=None, header=None)
    #go through each sheet to check which one is not empty
    if df.empty:
        for i in range(2,10):
            df = xl.parse("Sheet"+str(i), index_col=None, header=None)
            if not df.empty:
                break

    if isHeader(df.iloc[0][0]):
        df.drop([0],inplace=True)
        df.reset_index(inplace=True, drop=True)
    
    #remove extra col or row
    df.dropna(how="all",axis=1,inplace=True)
    df.dropna(how="all",axis=0,inplace=True)
    
    #after dropping all NaN row and col, it should have either 3 or 4 cols
    try:
        if len(df.columns)==3:
            df.columns = ['timestamp','name','text']
        if len(df.columns)==4:
            df.columns = ['timestamp','name','text','translation']
    except:
        print("Number of columns is weird go check")
        
    base_name = os.path.splitext(file_name)[0]
    df.loc[0]=[base_name,"",""]
    df.to_csv(save_name, sep=';', index=False, header=False)
    
    #running batch file to pop up text and inqscribe, edit path if needed
    batfile = open('reformat.bat','w+')
    batfile.write(r'start "" /D "C:\Program Files (x86)\InqScribe" "inqscribe.exe"')
    batfile.write("\n")
    batfile.write(r'start "" /D "%~dp0" '+save_name) #run txt file where the bat file is located
    batfile.close()
    p = Popen("reformat.bat")
    stdout, stderr = p.communicate()

filelist = ["190911_CM SH YoscarTable Yoscar Samantha Master Transcript.xlsx","191003_SH_SM-Magic Moment.xlsx"]

for file in filelist:
    reformat(file)